## Import library

In [34]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.applications import MobileNetV2, InceptionResNetV2, VGG16
from keras.preprocessing import image  # Modified import
from keras.preprocessing.image import ImageDataGenerator


In [55]:
import os
import warnings
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Activation,Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from keras.optimizers import Adam
from tensorflow.keras.models import load_model
import PIL
from PIL import Image
import io
from tensorflow.keras.applications.vgg16 import VGG16
from sklearn.metrics import classification_report,confusion_matrix
from tensorflow.keras.models import Model


In [54]:
img_dataset = "bangkit/"

In [26]:
BATCH_SIZE=32
IMG_SIZE=(244,244)

In [40]:
Generator=ImageDataGenerator(rescale=1/255.0,
                                 zoom_range=0.2,
                                 shear_range=0.3,
                                 horizontal_flip=True,
                                 validation_split=0.2,
                                 brightness_range=[0.5,1.5])                                 

In [41]:
train_ds=Generator.flow_from_directory(
    img_dataset,
    class_mode='binary',
    shuffle=True,
    batch_size=BATCH_SIZE,
    target_size=IMG_SIZE,
    subset="training"
)

validation_ds=Generator.flow_from_directory(
    img_dataset,
    class_mode='binary',
    shuffle=True,
    batch_size=BATCH_SIZE,
    target_size=IMG_SIZE,
    subset="validation"
)

Found 961 images belonging to 2 classes.
Found 240 images belonging to 2 classes.


In [29]:
class_names = list(train_ds.class_indices.keys())
class_names

['earmites', 'konjungtivitis']

In [36]:
### Ploting Augmented Images
def plot_random_images():
    images,labels=train_ds.next()
    plt.figure(figsize=(20,10))
    for i in range(10):
        plt.subplot(2,5,i+1)
        plt.imshow(images[i])
        plt.xticks([])
        plt.yticks([])
        plt.xlabel(image_class_dict[labels[i]])

In [42]:
# plot_random_images()

## Model Making

In [46]:
Epoch = 5

In [47]:
model = tf.keras.models.Sequential([
    # Menambahkan blok-blok konvolusi
    Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2), strides=(2, 2)),

    Conv2D(128, (3, 3), activation='relu', padding='same'),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2), strides=(2, 2)),

    Conv2D(256, (3, 3), activation='relu', padding='same'),
    Conv2D(256, (3, 3), activation='relu', padding='same'),
    Conv2D(256, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2), strides=(2, 2)),

    # Konversi matriks fitur menjadi vektor
    Flatten(),

    # Menambahkan layer-layer fully connected
    Dense(2046, activation='relu'),
    Dense(2046, activation='relu'),

    Dense(2, activation='softmax')

])

model.compile(optimizer="adam",loss="binary_crossentropy",metrics="accuracy")

In [45]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 64)     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 112, 112, 128)     73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 112, 112, 128)     147584    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 56, 56, 128)      0         
 2D)                                                    

In [ ]:
history = model.fit_generator(
    train_ds,
    validation_data=validation_ds,
    epochs=Epoch,
    validation_steps=validation_ds.samples // validation_ds.batch_size,
    steps_per_epoch=train_ds.samples // train_ds.batch_size,
    verbose = 1
)